In [1]:
import pandas as pd
import pyarrow as pa
import json
from datetime import date, datetime

In [5]:
!head flights-10m.csv

FlightDate,DepTime,DepDelay,ArrTime,ArrDelay,AirTime,Distance
2006-01-01,0905,5.00,1229,19.00,350.00,2475.00
2006-01-02,1147,167.00,1546,216.00,343.00,2475.00
2006-01-03,0853,-7.00,1208,-2.00,344.00,2475.00
2006-01-04,0855,-5.00,1157,-13.00,331.00,2475.00
2006-01-05,0857,-3.00,1153,-17.00,321.00,2475.00
2006-01-06,0856,-4.00,1138,-32.00,320.00,2475.00
2006-01-08,0857,-3.00,1208,-2.00,346.00,2475.00
2006-01-09,0903,3.00,1210,0.00,334.00,2475.00
2006-01-10,0853,-7.00,1149,-21.00,334.00,2475.00


In [6]:
!head flights-3m.csv

FL_DATE,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE
20060101,0905,5,1229,19,350,2475
20060102,1147,167,1546,216,343,2475
20060103,0853,-7,1208,-2,344,2475
20060104,0855,-5,1157,-13,331,2475
20060105,0857,-3,1153,-17,321,2475
20060106,0856,-4,1138,-32,320,2475
20060108,0857,-3,1208,-2,346,2475
20060109,0903,3,1210,0,334,2475
20060110,0853,-7,1149,-21,334,2475


In [7]:
def json_serial(obj):
    return int(obj.strftime("%s"))

def build_arrow(name, with_json):
    data = pd.read_csv(f'{name}.csv', encoding='utf-8', dtype={'FL_DATE': 'str', 'ARR_TIME': 'str', 'DEP_TIME': 'str'})
    data = data.dropna()

    arr_time = data.FL_DATE + data.ARR_TIME.replace('2400', '0000')
    data['ARRIVAL'] = pd.to_datetime(arr_time, format='%Y%m%d%H%M')

    dep_time = data.FL_DATE + data.DEP_TIME.replace('2400', '0000')
    data['DEPARTURE'] = pd.to_datetime(dep_time, format='%Y%m%d%H%M')

    data = data.drop(['FL_DATE', 'ARR_TIME', 'DEP_TIME'], axis=1)
    if 'ORIGIN' in data.columns:
        data = data.drop(['ORIGIN', 'DEST'], axis=1)

    df = data.dropna()

    if with_json:
        d = {}
        for column in df.columns:
            d[column]=list(df[column])

        with open(f'{name}.json', 'w') as f:
            json.dump(d, f, default=json_serial, separators=(',', ':'))

    df2 = df.astype({'DEP_DELAY': 'int16', 'ARR_DELAY': 'int16', 'AIR_TIME': 'int16', 'DISTANCE': 'int16'})

    table = pa.Table.from_pandas(df2)

    # table = table.column('ARRIVAL').cast(pa.TimestampValue, True)

    writer = pa.RecordBatchFileWriter(f'{name}.arrow', table.schema)
    writer.write(table)
    writer.close()
    
# build_arrow('flights-10m', False)

In [14]:
data = pd.read_csv(f'flights-10m.csv', encoding='utf-8', dtype={'FlightDate': 'str', 'ArrTime': 'str', 'DepTime': 'str'})
data = data.dropna()

data.head()

,FlightDate,DepTime,DepDelay,ArrTime,ArrDelay,AirTime,Distance
0,2006-01-01,0905,5.0,1229,19.0,350.0,2475.0
1,2006-01-02,1147,167.0,1546,216.0,343.0,2475.0
2,2006-01-03,0853,-7.0,1208,-2.0,344.0,2475.0
3,2006-01-04,0855,-5.0,1157,-13.0,331.0,2475.0
4,2006-01-05,0857,-3.0,1153,-17.0,321.0,2475.0


In [16]:
data = data.rename(index=str, columns={"FlightDate": "FL_DATE", "DepTime": "DEP_TIME", "ArrTime": "ARR_TIME", "Distance": "DISTANCE", "AirTime": "AIR_TIME", "DepDelay": "DEP_DELAY", "ArrDelay": "ARR_DELAY"})

arr_time = data.FL_DATE + data.ARR_TIME.replace('2400', '0000')
data['ARRIVAL'] = pd.to_datetime(arr_time, format='%Y-%m-%d%H%M')

dep_time = data.FL_DATE + data.DEP_TIME.replace('2400', '0000')
data['DEPARTURE'] = pd.to_datetime(dep_time, format='%Y-%m-%d%H%M')

data = data.drop(['FL_DATE', 'ARR_TIME', 'DEP_TIME'], axis=1)
if 'ORIGIN' in data.columns:
    data = data.drop(['ORIGIN', 'DEST'], axis=1)

df = data.dropna()

df2 = df.astype({'DEP_DELAY': 'int16', 'ARR_DELAY': 'int16', 'AIR_TIME': 'int16', 'DISTANCE': 'int16'})

table = pa.Table.from_pandas(df2)

# table = table.column('ARRIVAL').cast(pa.TimestampValue, True)

writer = pa.RecordBatchFileWriter(f'flights-10m.arrow', table.schema)
writer.write(table)
writer.close()

In [8]:
for name in ['flights-10k', 'flights-200k', 'flights-500k', 'flights-1m', 'flights-3m', 'flights-10m']:
    print(name)
    
    build_arrow(name, False)

flights-10k
flights-200k


In [7]:
!ls -lah

total 717528
drwxr-xr-x@ 20 domoritz  staff   640B Jun 11 18:06 .
drwxr-xr-x  24 domoritz  staff   768B Jun 10 19:59 ..
-rw-r--r--@  1 domoritz  staff   6.0K Nov 29  2017 .DS_Store
drwxr-xr-x   4 domoritz  staff   128B Jun  8 22:36 .ipynb_checkpoints
-rw-r--r--@  1 domoritz  staff    28M Nov 28  2017 564230852_T_ONTIME.csv
-rw-r--r--@  1 domoritz  staff   6.3M Nov 28  2017 564230852_T_ONTIME.zip
-rw-r--r--   1 domoritz  staff   4.5K Jun 11 18:06 convert.ipynb
-rw-r--r--   1 domoritz  staff   316K Jun 11 18:06 flights-10k.arrow
-rw-r--r--   1 domoritz  staff   400K Nov 29  2017 flights-10k.csv
-rw-r--r--   1 domoritz  staff   409K Jun  8 22:39 flights-10k.json
-rw-r--r--   1 domoritz  staff    30M Jun  8 22:44 flights-1m.arrow
-rw-r--r--   1 domoritz  staff    31M Nov 29  2017 flights-1m.csv
-rw-r--r--   1 domoritz  staff   5.8M Jun 11 15:06 flights-200k.arrow
-rw-r--r--   1 domoritz  staff   7.7M Feb 13 21:44 flights-200k.csv
-rw-r--r--   1 domoritz  staff   7.9M Jun  8 22:40 flights-2